# Week 3 Final Assignment
Hideyuki SAKONO

## 1 Scrape a web page and make a list of neighborhoods in Toronto

In [1]:
!pip install beautifulsoup4 requests
!pip install geopy
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import requests
from bs4 import BeautifulSoup

#test connection
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## 2 Make the list updated with coordinates

In [3]:
coordinates_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
coordinates_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
toronto_data=df
toronto_data = toronto_data.join(coordinates_data.set_index('Postal Code'), on='PostalCode')
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## 3 Explore and cluster the neighborhoods in Toronto

In [5]:
# First of all, just plotting the points according to ZIP codes
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['PostalCode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [6]:
# Same as the example of New York city, get the venue data from foursquare.com

CLIENT_ID = 'MIGLPULSS4IWLZ25BAOH3GKBYQ5JHW4Y110G25JME3LDGMP0' # your Foursquare ID
CLIENT_SECRET = 'D0YTIBRKNXDTUEPOECR20D5YH4F0AMRRUOUVOP5ZUGGV52C2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
toronto_venues = getNearbyVenues(names=toronto_data['PostalCode'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [9]:
print(toronto_venues.shape)
toronto_venues.head()

(1992, 7)


,PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [10]:
toronto_venues.groupby('PostalCode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,1,1,1,1,1,1
M1C,1,1,1,1,1,1
M1E,9,9,9,9,9,9
M1G,3,3,3,3,3,3
M1H,7,7,7,7,7,7
M1J,1,1,1,1,1,1
M1K,5,5,5,5,5,5
M1L,9,9,9,9,9,9
M1M,2,2,2,2,2,2


## Looking at the number of venues above, including the data with small number of venues does not make sense if we conduct clustering the data as same as the NY city assignment.
## I will filter the data that have more than 20 venues.

In [11]:
toronto_venues.groupby('PostalCode').count()
zipcode_del = (toronto_venues.groupby('PostalCode').count()["Venue"] < 20)
toronto_data_filtered =pd.merge(toronto_data, zipcode_del, how="inner" ,on="PostalCode")
toronto_data_new=toronto_data_filtered[toronto_data_filtered['Venue'] == False].drop("Venue", axis=1)
toronto_data_new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
22,M4G,East York,Leaside,43.709060,-79.363452
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
29,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
32,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
35,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [12]:
# making venues data again based on the filtered list
toronto_venues = getNearbyVenues(names=toronto_data_new['PostalCode'],
                                   latitudes=toronto_data_new['Latitude'],
                                   longitudes=toronto_data_new['Longitude']
                                  )

M5A
M7A
M5B
M5C
M5E
M4G
M5G
M5H
M2J
M5J
M6J
M4K
M5K
M6K
M5L
M4M
M5M
M2N
M4S
M5S
M6S
M5T
M5W
M4X
M5X
M4Y


## Based on the filtered data above, make cluster and map them

In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M2J,0.016129,0.0,0.0,0.016129,0.0,0.0,0.0,0.000000,0.016129,0.0,0.0,0.032258,0.032258,0.016129,0.016129,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00000,0.00000,0.016129,0.016129,0.016129,0.0,0.000000,0.0,0.00000,0.000000,0.016129,0.0,0.0,0.0,0.129032,0.0,0.080645,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.032258,0.016129,0.0,0.0,0.0,0.0,0.0,0.016129,0.016129,0.0,0.0,0.016129,0.0,0.0,0.0,0.016129,0.0,0.0,0.0,0.0,0.0,0.064516,0.00,0.000000,0.000000,0.032258,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.016129,0.000000,0.032258,0.0,0.032258,0.00000,0.0,0.000000,0.0,0.000000,0.016129,0.016129,0.00000,0.016129,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.016129,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.016129,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.016129,0.032258,0.0,0.0,0.016129,0.000000,0.0,0.000000,0.016129,0.016129,0.0,0.0,0.0,0.0,0.00000,0.0,0.016129,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.016129,0.000000,0.0,0.016129,0.0,0.0,0.0,0.000000,0.016129,0.00000,0.0

In [15]:
num_top_venues = 5

for hood in toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M2J----
                  venue  freq
0        Clothing Store  0.13
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.06
3            Restaurant  0.03
4            Food Court  0.03


----M2N----
                 venue  freq
0          Coffee Shop  0.15
1     Ramen Restaurant  0.11
2          Pizza Place  0.07
3  Japanese Restaurant  0.07
4                 Bank  0.04


----M4G----
                 venue  freq
0          Coffee Shop  0.12
1  Sporting Goods Shop  0.12
2       Breakfast Spot  0.08
3     Sushi Restaurant  0.08
4         Burger Joint  0.08


----M4K----
                venue  freq
0    Greek Restaurant  0.24
1  Italian Restaurant  0.07
2         Coffee Shop  0.07
3      Ice Cream Shop  0.05
4         Yoga Studio  0.02


----M4M----
            venue  freq
0     Coffee Shop  0.11
1          Bakery  0.07
2            Café  0.07
3       Gastropub  0.07
4  Ice Cream Shop  0.04


----M4S----
              venue  freq
0       Pizza Place  0.11
1  Sushi Restaurant  0.07


In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2J,Clothing Store,Coffee Shop,Fast Food Restaurant,Bank,Restaurant,Convenience Store,Japanese Restaurant,Jewelry Store,Bakery,Food Court
1,M2N,Coffee Shop,Ramen Restaurant,Japanese Restaurant,Pizza Place,Grocery Store,Plaza,Shopping Mall,Sandwich Place,Café,Korean Restaurant
2,M4G,Coffee Shop,Sporting Goods Shop,Breakfast Spot,Burger Joint,Sushi Restaurant,Bank,Pet Store,Dessert Shop,Mexican Restaurant,Supermarket
3,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop,Sandwich Place,Café,Restaurant,Caribbean Restaurant,Pub
4,M4M,Coffee Shop,Café,Gastropub,Bakery,Latin American Restaurant,Furniture / Home Store,Neighborhood,Park,Comfort Food Restaurant,Middle Eastern Restaurant
5,M4S,Pizza Place,Gym,Coffee Shop,Sandwich Place,Dessert Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Discount Store,Japanese Restaurant
6,M4X,Pizza Place,Coffee Shop,Italian Restaurant,Café,Restaurant,Pub,Pharmacy,Chinese Restaurant,Bakery,Plaza
7,M4Y,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Gay Bar,Burrito Place,Indian Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Gym
8,M5A,Coffee Shop,Bakery,Pub,Park,Café,Discount Store,Dessert Shop,Beer Store,French Restaurant,Mexican Restaurant
9,M5B,Coffee Shop,Sandwich Place,Clothing Store,Café,Bank,Cosmetics Shop,Pizza Place,Thai Restaurant,Hotel,Japanese Restaurant


In [18]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 4, 3, 2, 0, 2, 2, 2, 2], dtype=int32)

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M2J,Clothing Store,Coffee Shop,Fast Food Restaurant,Bank,Restaurant,Convenience Store,Japanese Restaurant,Jewelry Store,Bakery,Food Court
1,2,M2N,Coffee Shop,Ramen Restaurant,Japanese Restaurant,Pizza Place,Grocery Store,Plaza,Shopping Mall,Sandwich Place,Café,Korean Restaurant
2,4,M4G,Coffee Shop,Sporting Goods Shop,Breakfast Spot,Burger Joint,Sushi Restaurant,Bank,Pet Store,Dessert Shop,Mexican Restaurant,Supermarket
3,3,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop,Sandwich Place,Café,Restaurant,Caribbean Restaurant,Pub
4,2,M4M,Coffee Shop,Café,Gastropub,Bakery,Latin American Restaurant,Furniture / Home Store,Neighborhood,Park,Comfort Food Restaurant,Middle Eastern Restaurant
5,0,M4S,Pizza Place,Gym,Coffee Shop,Sandwich Place,Dessert Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Discount Store,Japanese Restaurant
6,2,M4X,Pizza Place,Coffee Shop,Italian Restaurant,Café,Restaurant,Pub,Pharmacy,Chinese Restaurant,Bakery,Plaza
7,2,M4Y,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Gay Bar,Burrito Place,Indian Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Gym
8,2,M5A,Coffee Shop,Bakery,Pub,Park,Café,Discount Store,Dessert Shop,Beer Store,French Restaurant,Mexican Restaurant
9,2,M5B,Coffee Shop,Sandwich Place,Clothing Store,Café,Bank,Cosmetics Shop,Pizza Place,Thai Restaurant,Hotel,Japanese Restaurant


In [20]:
toronto_merged = toronto_data_new
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Café,Discount Store,Dessert Shop,Beer Store,French Restaurant,Mexican Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Gym,Japanese Restaurant,Mexican Restaurant,Diner,Park,College Auditorium
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Sandwich Place,Clothing Store,Café,Bank,Cosmetics Shop,Pizza Place,Thai Restaurant,Hotel,Japanese Restaurant
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Restaurant,Clothing Store,Diner,Beer Bar,Seafood Restaurant,Farmers Market
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Vegetarian / Vegan Restaurant,Seafood Restaurant,Farmers Market,Beer Bar,Indian Restaurant,Spa
22,M4G,East York,Leaside,43.709060,-79.363452,4,Coffee Shop,Sporting Goods Shop,Breakfast Spot,Burger Joint,Sushi Restaurant,Bank,Pet Store,Dessert Shop,Mexican Restaurant,Supermarket
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Burger Joint,Salad Place,Restaurant,Pizza Place
29,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2,Coffee Shop,Café,Sandwich Place,Clothing Store,Gym,Restaurant,Sushi Restaurant,Concert Hall,Deli / Bodega,Japanese Restaurant
32,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2,Clothing Store,Coffee Shop,Fast Food Restaurant,Bank,Restaurant,Convenience Store,Japanese Restaurant,Jewelry Store,Bakery,Food Court
35,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,2,Coffee Shop,Hotel,Café,Pizza Place,Aquarium,Scenic Lookout,Sandwich Place,Brewery,Italian Restaurant,Deli / Bodega


In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

### As a result, we should manipulate some parameters such as distance from each point and number of most common venue, for example five instead of ten.